In [1]:
cd ..

/home/compute/APS-360


In [2]:
import torch
import torch.nn as nn
import pandas as pd
from common import model
from common import evaluate
from common import separate_test
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from ann import *

In [3]:
def day_to_num(day):
    ls = ['sunday', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'other']
    if day != 0:
        return ls.index(day) + 1
    return 0

def time_to_num(time):
    if time == 0:
        return 0
    time = time.replace(':', '')
    return float(time)

In [4]:
df_data = pd.read_csv('new_test_animes_data.csv')
df_data = df_data.fillna(0)
len(df_data.columns)

45

In [5]:
df_data['broadcast_day_of_the_week'] = df_data['broadcast_day_of_the_week'].apply(day_to_num)
df_data['broadcast_start_time'] = df_data['broadcast_start_time'].apply(time_to_num)

In [6]:
df_old = pd.read_csv('test_data')

for column in df_old.columns:
    if column not in df_data.columns:
        df_data[column] = 0

In [7]:
df_data = df_data.fillna(0)

keep = ['popularity', 'broadcast_day_of_the_week', 'average_episode_duration', 'broadcast_start_time']
for d in df_data.columns:
    if 'id' in d:
        keep.append(d)

df_data = df_data[keep]
df_data= df_data.drop(labels=['id'], axis=1)

In [8]:
df_data

,popularity,broadcast_day_of_the_week,average_episode_duration,broadcast_start_time,genres_0_id,genres_1_id,genres_2_id,genres_3_id,studios_0_id,genres_4_id,...,genres_6_id,genres_7_id,studios_1_id,studios_2_id,genres_8_id,genres_9_id,genres_10_id,studios_3_id,studios_4_id,studios_5_id
0,19,1,1440,930.0,1,2.0,10.0,27.0,18,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
1,285,5,1435,2356.0,1,10.0,23.0,27.0,569,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
2,528,1,1447,1700.0,1,50.0,4.0,14.0,2674,42.0,...,76.0,41.0,0.0,0.0,0,0,0,0,0,0
3,609,6,1380,2100.0,1,2.0,8.0,10.0,290,62.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
4,662,7,1420,2300.0,1,53.0,4.0,27.0,858,0.0,...,0.0,0.0,1835.0,0.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,19880,8,300,0.0,15,36.0,0.0,0.0,-1,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
116,20162,7,300,1755.0,4,15.0,0.0,0.0,1067,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
117,20561,0,300,0.0,15,36.0,0.0,0.0,2647,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
118,20617,0,0,0.0,1,2.0,4.0,15.0,-1,31.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0


In [9]:
df_data.to_csv('new_test_data')

In [10]:
for d in df_data.columns:
    print(d, df_data[d].dtype)

popularity int64
broadcast_day_of_the_week int64
average_episode_duration int64
broadcast_start_time float64
genres_0_id int64
genres_1_id float64
genres_2_id float64
genres_3_id float64
studios_0_id int64
genres_4_id float64
genres_5_id float64
genres_6_id float64
genres_7_id float64
studios_1_id float64
studios_2_id float64
genres_8_id int64
genres_9_id int64
genres_10_id int64
studios_3_id int64
studios_4_id int64
studios_5_id int64


In [11]:
model = ANN('anneval', 64, 0.01, 75, 8, 32, F.hardswish)
model_path = 'training/ANNf_64_0.01_75_8_32_hardswish_71'
state = torch.load(model_path)
model.load_state_dict(state)

<All keys matched successfully>

In [12]:
test = get_data_loaders('new_test_data', 64)
evaluate.evaluate(model, test)

/home/compute/APS-360/ann/ann.py:54: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data = torch.tensor(row[2:]).float()


0.2849285875202564